In [15]:
pip install rioxarray

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Obtaining dependency information for rioxarray from https://files.pythonhosted.org/packages/4e/34/8ee402df8a49228b6aae89ab2127cdd5aa0ba0976904af7d55270f4f30b8/rioxarray-0.15.0-py3-none-any.whl.metadata
   ---------------------------------------- 53.7/53.7 kB 395.1 kB/s eta 0:00:00


In [26]:
import os 
import arcpy
import datetime
import xarray as xr
import pandas as pd
import glob
import rioxarray
import rasterio
import numpy as np

In [27]:
filenames = glob.glob("G:/usgrids2020/raster/AL/*.tif")

filenames[1][25:-4]
fields = [field[25:-4] for field in filenames]
fields

['AL_A000_004BT_CNT',
 'AL_A000_004FT_CNT',
 'AL_A000_004MT_CNT',
 'AL_A005_009BT_CNT',
 'AL_A005_009FT_CNT',
 'AL_A005_009MT_CNT',
 'AL_A010_014BT_CNT',
 'AL_A010_014FT_CNT',
 'AL_A010_014MT_CNT',
 'AL_A015_019BT_CNT',
 'AL_A015_019FT_CNT',
 'AL_A015_019MT_CNT',
 'AL_A020_024BT_CNT',
 'AL_A020_024FT_CNT',
 'AL_A020_024MT_CNT',
 'AL_A025_029BT_CNT',
 'AL_A025_029FT_CNT',
 'AL_A025_029MT_CNT',
 'AL_A030_034BT_CNT',
 'AL_A030_034FT_CNT',
 'AL_A030_034MT_CNT',
 'AL_A035_039BT_CNT',
 'AL_A035_039FT_CNT',
 'AL_A035_039MT_CNT',
 'AL_A040_044BT_CNT',
 'AL_A040_044FT_CNT',
 'AL_A040_044MT_CNT',
 'AL_A045_049BT_CNT',
 'AL_A045_049FT_CNT',
 'AL_A045_049MT_CNT',
 'AL_A050_054BT_CNT',
 'AL_A050_054FT_CNT',
 'AL_A050_054MT_CNT',
 'AL_A055_059BT_CNT',
 'AL_A055_059FT_CNT',
 'AL_A055_059MT_CNT',
 'AL_A060_064BT_CNT',
 'AL_A060_064FT_CNT',
 'AL_A060_064MT_CNT',
 'AL_A065_069BT_CNT',
 'AL_A065_069FT_CNT',
 'AL_A065_069MT_CNT',
 'AL_A070_074BT_CNT',
 'AL_A070_074FT_CNT',
 'AL_A070_074MT_CNT',
 'AL_A075_

In [28]:

time = "2020"
# List of GeoTIFF filenames

# Create a time dimension from the filenames
combined_data = xr.concat([rioxarray.open_rasterio(f, chunks={'x': 5490, 'y': 5490}) for f in filenames],dim = time, data_vars = fields)
# chunks = {'x': 5490, 'y': 5490, 'band': len(fields)}
# da = xr.concat([rioxarray.open_rasterio(f, chunks=chunks, variable = fields, band_as_variable = True) for f in filenames], dim=time)

ValueError: data_vars is not a valid argument when concatenating DataArray objects

In [113]:
combined_data.to_netcdf("G:/usgrids2020/multidimensional_data2.nc")

In [127]:
combined_data

<xarray.DataArray (concat_dim: 89, band: 1, y: 1168, x: 861)>
dask.array<concatenate, shape=(89, 1, 1168, 861), dtype=float64, chunksize=(1, 1, 1168, 861), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int32 1
  * x            (x) float64 -88.47 -88.47 -88.46 -88.46 ... -84.9 -84.89 -84.89
  * y            (y) float64 35.01 35.0 35.0 34.99 ... 30.16 30.15 30.15 30.14
    spatial_ref  int32 0
  * concat_dim   (concat_dim) object 'AL_A000_004BT_CNT' ... 'AL_WOMCHILD_CNT'
Attributes: (12/13)
    AREA_OR_POINT:           Area
    TIFFTAG_SOFTWARE:        GDAL 3.7.0e (3070046) (4.5.1;1700;3.0.0;2.0.7;;;...
    STATISTICS_COUNT:        755005.0
    STATISTICS_MAXIMUM:      193.63690035468
    STATISTICS_MEAN:         0.3795060959757
    STATISTICS_MEDIAN:       nan
    ...                      ...
    STATISTICS_SKIPFACTORX:  1
    STATISTICS_SKIPFACTORY:  1
    STATISTICS_STDDEV:       1.8456040028011
    _FillValue:              -1.7976931348623157e+308
    scale_factor:            1.0
    add_offset:              0.0

In [40]:
arcpy.env.workspace = "G:/usgrids2020/final_gdbs"
gdbs = arcpy.ListWorkspaces("*", "FILEGDB")
procList = [os.path.join(gdb,f"{os.path.basename(gdb)[:-4]}_joined") for gdb in gdbs]
fcInMem = arcpy.management.CopyFeatures(procList[9], "in_memory/fcInMem")
#Create the temporary id for joining fields later
arcpy.AddField_management(fcInMem,'tempid','LONG')
arcpy.CalculateField_management(fcInMem,"tempid",'!OBJECTID!','PYTHON')
#Calculate the area(land+water) by GEODESIC method => Kytt recommended
arcpy.management.CalculateGeometryAttributes(fcInMem, "GEODESIC_AREA AREA_GEODESIC", "", "SQUARE_METERS")

with arcpy.da.SearchCursor(fcInMem, ["GEODESIC_AREA", "tempid"]) as cursor:
    for i in range(10):
        for row in cursor:
            print(row)
            break

(120458.14118966038, 1, 120458.14118966038, 1)
(6363.599517613925, 2, 6363.599517613925, 2)
(20116.06609771172, 3, 20116.06609771172, 3)
(1051291.3000202314, 4, 1051291.3000202314, 4)
(18467.461655443298, 5, 18467.461655443298, 5)
(7352.219540517548, 6, 7352.219540517548, 6)
(1716.4800594952872, 7, 1716.4800594952872, 7)
(1064109.3654185396, 8, 1064109.3654185396, 8)
(19993.87506006442, 9, 19993.87506006442, 9)
(32972.06493657967, 10, 32972.06493657967, 10)


In [38]:
a = os.path.basename(procList[8])
a

'usade_joined'

In [39]:
procList[8]

'G:/usgrids2020/final_gdbs\\usade.gdb\\usade_joined'